# 페이스북 크롤링

In [245]:
# 페이지 스크롤 횟수 설정
num_scrolldown = 10

In [246]:
from bs4 import BeautifulSoup as bs
import time
import random
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [247]:
import pandas as pd

## 크롤링할 사이트 접속

In [248]:
# 크롤링할 웹사이트 주소
url = 'https://www.facebook.com/GyeonggiKorea.en'

# webdriver 로드
driver = webdriver.Chrome('chromedriver')
driver.get(url)
time.sleep(3)
driver.implicitly_wait(5)

# 창 최대화
driver.maximize_window()
time.sleep(1)
driver.implicitly_wait(5)

## 페이지 스크롤 다운

In [249]:
# 페이지 스크롤 다운
for i in range(num_scrolldown):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    time.sleep( random.uniform(1,3) )

## 크롤링

In [250]:
# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
print(soup.title.text)

# 글 날짜 수집
info_dates_span = soup.find_all('a', 'oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw')

# 글 내용 크롤링
content_pages = soup.find_all('div', class_='kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q')

# 좋아요, 댓글, 공유 수 정보 나온 페이지 수집
info_pages = soup.find_all('div', 'bp9cbjyn m9osqain j83agx80 jq4qci2q bkfpd7mw a3bd9o3v kvgmc6g5 wkznzc2l oygrvhab dhix69tm jktsbyx5 rz4wbd8a osnr6wyh a8nywdso s1tcr66n')

print("날짜 수집 개수 : ", len(info_dates_span))
print("글 내용 수집 개수 : ", len(content_pages))
print("각종 정보 수집 개수 : ", len(info_pages))

Gyeonggi Korea | Facebook
날짜 수집 개수 :  52
글 내용 수집 개수 :  51
각종 정보 수집 개수 :  51


## 글 날짜, 글 내용, 좋아요 수, 댓글 수, 공유 수 리스트화

### 날짜 수집

In [251]:
# 날짜 수집
num_date_list = []
for date in info_dates_span:
    if date.span != None:
        temp = date.span.text
        num_date_list.append(temp)

print('날짜 리스트 원소수 : ',len(num_date_list))

날짜 리스트 원소수 :  51


In [252]:
# 날짜 형식 변환해서 리스트로 저장
mode_num_date_list = []
for date in num_date_list:
    if '시간' in date:
        temp = datetime.today().strftime("%Y-%m-%d")
        mode_num_date_list.append(temp)
    elif '어제' in date:
        temp = datetime.today().strftime("%Y-%m-%d")
        temp = temp.split('-')
        year = temp[0]
        month = temp[1]
        day = int( temp[2] ) - 1
        temp_date = f'{year}-{month:0>2}-{day:0>2}'
        mode_num_date_list.append(temp_date)
    else:
        temp = date.split('일')[0]
        temp = temp.replace(' ', '')
        temp = temp.replace('년', '-')
        temp = temp.replace('월', '-')
        temp_li = temp.split('-')
        if len(temp_li) == 3 :
            year = temp_li[0]
            month = temp_li[1]
            day = temp_li[2]
            temp_date = f'{year}-{month:0>2}-{day:0>2}'
            mode_num_date_list.append(temp_date)
        else:
            year = datetime.today().strftime("%Y")
            month = temp_li[0]
            day = temp_li[1]
            temp_date = f'{year}-{month:0>2}-{day:0>2}'
            mode_num_date_list.append(temp_date)

print("변환한 날짜 리스트 원소수 : ", len(mode_num_date_list) )

변환한 날짜 리스트 원소수 :  51


### 글 내용, 좋아요 수, 댓글 수, 공유 수 수집

In [253]:
# 글 내용 수집
contents_list = []
for content in content_pages:
    temp = content.text.split('.')[0]
    contents_list.append(temp)
    
print('글 내용 리스트 원소수 : ',len(contents_list))

글 내용 리스트 원소수 :  51


In [254]:
# 조회수 수집
num_view_list = []
for i in range(len(contents_list)):
    num_view_list.append(0)

print('조회수 리스트 원소수 : ',len(num_view_list))

조회수 리스트 원소수 :  51


In [255]:
# 좋아요 수 수집
num_like_list = []
for like in info_pages:
    temp = like.find('span', 'pcp91wgn').text
    if '천' in temp:
        num_like_list.append(temp)
    else:
        temp = int(temp)
        num_like_list.append(temp)        
    
print('좋아요 수 리스트 원소수 : ',len(num_like_list))

좋아요 수 리스트 원소수 :  51


In [256]:
# 댓글 수, 공유 수 수집
num_comment_list = []
num_share_list = []
for comment in info_pages:
    temp = comment.find_all('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain')
    if len(temp) == 1:
        for one in temp:
            if '댓글' in one.text:
                comment_one = int(one.text[3:-1])
                num_comment_list.append(comment_one)
                num_share_list.append(0)
            elif '공유' in one.text:
                comment_share = int(one.text[3:-1])
                num_share_list.append(comment_share)
                num_comment_list.append(0)
    elif len(temp) == 2:
        for one in temp:
            if '댓글' in one.text:
                comment_one = int(one.text[3:-1])
                num_comment_list.append(comment_one)
            elif '공유' in one.text:
                comment_share = int(one.text[3:-1])
                num_share_list.append(comment_share)
    else:
        num_share_list.append(0)
        num_comment_list.append(0)

print('공유 수 리스트 원소수 : ',len(num_share_list))
print('댓글 수 리스트 원소수 : ',len(num_comment_list))

공유 수 리스트 원소수 :  51
댓글 수 리스트 원소수 :  51


### 게시글 링크 수집

In [266]:
# 게시글 링크 수집
link_list = []
raw_link = soup.find_all('div', class_='du4w35lb k4urcfbm l9j0dhe7 sjgh65i0')
for link in raw_link:
    temp_link = link.find('a', class_='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw').attrs['href'].split('?')[0]
    link_list.append(temp_link)
print('링크 리스트 원소수 : ',len(link_list))

링크 리스트 원소수 :  51


In [267]:
link_list[:3]

['https://www.facebook.com/GyeonggiKorea.en/posts/pfbid02esMBTuC5fkcY7F1vHskb7g1qWip1LzEViYYUx6aDwyZ4Y9sFiVDCKXB2SVmwokb4l',
 'https://www.facebook.com/GyeonggiKorea.en/posts/pfbid0wrBX1VahhLmzASsJ9CEzQbpaVn8VF8Y19S5KNGgP8xrN3QQMqPFEC72RdYG1EBpl',
 'https://www.facebook.com/GyeonggiKorea.en/posts/pfbid0Jnx9UH7AprfVdbYCTQCR3jecPHBrPK2ux3MtrPZAvY9E5DPXYU3HFwF4HSG5zmgWl']

In [259]:
# chromedriver 종료
driver.quit()

## 데이터 셋 만들기

In [268]:
# 수집 데이터를 데이터 셋으로 만들기
df = {'날짜':mode_num_date_list, '글_내용':contents_list, '조회수':num_view_list, '좋아요_수':num_like_list, '공유수':num_share_list, '댓글수':num_comment_list, '링크':link_list}
data = pd.DataFrame(df)
data

,날짜,글_내용,조회수,좋아요_수,공유수,댓글수,링크
0,2022-03-23,Gyeonggi Launches Blockchain-based Artist Vide...,0,8,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
1,2022-03-21,Introduction of Gyeonggi Lifestyle Apps in Var...,0,16,2,0,https://www.facebook.com/GyeonggiKorea.en/post...
2,2022-03-19,"This year, Gyeonggi Province is promoting an i...",0,24,3,2,https://www.facebook.com/GyeonggiKorea.en/post...
3,2022-03-18,Catch this week's #GTV English #News for the l...,0,11,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
4,2022-03-17,Successful Overseas Expansion by Local Startup...,0,17,1,0,https://www.facebook.com/GyeonggiKorea.en/post...
5,2022-03-16,[2022 Gyeonggi Foreign Social Media Reporters ...,0,35,5,0,https://www.facebook.com/GyeonggiKorea.en/post...
6,2022-03-15,Gyeonggi to produce and distribute tourism bro...,0,47,1,6,https://www.facebook.com/GyeonggiKorea.en/post...
7,2022-03-12,Gyeonggi Province’s Suwon City creates “Foreig...,0,23,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
8,2022-03-11,Catch this week's #GTV English #News for the l...,0,12,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
9,2022-03-10,Gyeonggi Province's Hanam City home to new LG-...,0,23,1,0,https://www.facebook.com/GyeonggiKorea.en/post...


## 데이터 셋. 날짜로 슬라이싱

In [269]:
data2 = data[data['날짜'] >= '2022-01-01']
data2

,날짜,글_내용,조회수,좋아요_수,공유수,댓글수,링크
0,2022-03-23,Gyeonggi Launches Blockchain-based Artist Vide...,0,8,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
1,2022-03-21,Introduction of Gyeonggi Lifestyle Apps in Var...,0,16,2,0,https://www.facebook.com/GyeonggiKorea.en/post...
2,2022-03-19,"This year, Gyeonggi Province is promoting an i...",0,24,3,2,https://www.facebook.com/GyeonggiKorea.en/post...
3,2022-03-18,Catch this week's #GTV English #News for the l...,0,11,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
4,2022-03-17,Successful Overseas Expansion by Local Startup...,0,17,1,0,https://www.facebook.com/GyeonggiKorea.en/post...
5,2022-03-16,[2022 Gyeonggi Foreign Social Media Reporters ...,0,35,5,0,https://www.facebook.com/GyeonggiKorea.en/post...
6,2022-03-15,Gyeonggi to produce and distribute tourism bro...,0,47,1,6,https://www.facebook.com/GyeonggiKorea.en/post...
7,2022-03-12,Gyeonggi Province’s Suwon City creates “Foreig...,0,23,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
8,2022-03-11,Catch this week's #GTV English #News for the l...,0,12,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
9,2022-03-10,Gyeonggi Province's Hanam City home to new LG-...,0,23,1,0,https://www.facebook.com/GyeonggiKorea.en/post...


## 필요한 데이터만 추출

In [270]:
# 작성 게시글만 추출
removed_words = ["Catch this week's", "["]

# 제거할 단어 들어있는 행 인덱스 찾기
temp = data2['글_내용'].to_list()
remove_index = []
for i, s in enumerate(temp):
    for one in removed_words:
        if one in s:
            remove_index.append(i)
remove_index

[3, 5, 8, 12, 15, 18, 21, 31, 35, 38]

In [271]:
# 중복 인덱스 제거
temp_idx_set = set(remove_index)
mod_remove_index = list(temp_idx_set)
mod_remove_index

[3, 35, 5, 38, 8, 12, 15, 18, 21, 31]

## 데이터 셋. 인덱스 내림차순으로 정렬

In [272]:
# 인덱스 리셋, 내림차순 정렬
data3 = data2.drop(mod_remove_index).reset_index(drop=True)
final_data = data3.sort_index(ascending=False)
final_data

,날짜,글_내용,조회수,좋아요_수,공유수,댓글수,링크
29,2022-01-01,Happy New Year! We hope you have a happy year...,0,815,20,34,https://www.facebook.com/GyeonggiKorea.en/post...
28,2022-01-05,How should you take care of dry skin in the wi...,0,372,20,0,https://www.facebook.com/GyeonggiKorea.en/post...
27,2022-01-07,Contactless payment service launched on Gyeong...,0,68,1,1,https://www.facebook.com/GyeonggiKorea.en/post...
26,2022-01-08,Gyeonggi-do’s Pyeongtaek is a certified child-...,0,773,8,12,https://www.facebook.com/GyeonggiKorea.en/post...
25,2022-01-13,"5 Metaverse Travel Spots in Siheung City, Gye...",0,572,12,7,https://www.facebook.com/GyeonggiKorea.en/post...
24,2022-01-15,Gyeonggi Province’s Anyang City runs Korea’s f...,0,590,16,6,https://www.facebook.com/GyeonggiKorea.en/post...
23,2022-01-17,Gyeonggi Province publishes K-culture report,0,440,15,8,https://www.facebook.com/GyeonggiKorea.en/post...
22,2022-01-20,Abandoned wastewater treatment plant transform...,0,77,3,3,https://www.facebook.com/GyeonggiKorea.en/post...
21,2022-01-24,Gyeonggi Peace Plaza: Filled with captivating ...,0,48,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
20,2022-01-26,Gyeonggi Happiness Shop achieves record-break...,0,46,0,0,https://www.facebook.com/GyeonggiKorea.en/post...


## 엑셀 파일로 출력

In [273]:
# 수집한 데이터 파일료 출력
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'facebook_' + file_name + '.xlsx'
path = './crawling_data/facebook/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

facebook_20220323-174634.xlsx 파일 생성 완료


# 코드 마지막